<a href="https://colab.research.google.com/github/marquesarthur/vanilla-bert-vs-huggingface/blob/main/hugging_face_keras_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on 



1.   https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379
2.   https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/
3.   https://huggingface.co/transformers/training.html#fine-tuning-with-keras




**problem statement:**


*   a developer has to inspect an **artifact X**
*   Within the artifact, only a portion of the text is relevant to **input task Y**
*   We ought to build a model that establishes relationships between **Y** and **sentences x ∈ X** 
*  The model must determine: **is x relevant to task Y**




<br>

___

*Example of a task and an annotated artifact:*

<br>

[<img src="https://i.imgur.com/Zj1317H.jpg">](https://i.imgur.com/Zj1317H.jpg)




* The coloured sentences are sentences annotated as relevant to the input task. 
* The warmer the color, the more annotators selected that portion of the text. 
* For simplicity, we process the data and used sentences 

<br>

___

*Ultimately, our data is a tuple representing:*


*   **text** = artifact sentence

*   **question** = task description

*   **source** = URL of the artifact

*   **category_index** = whether sentence is relevant [or not] for the input task

*   **weights** = number of participants who annotated sentence as relevant


<br>

___



In [1]:
# @title Install dependencies

# !pip install transformers
# %tensorflow_version 2.x

In [2]:
# !pip install scikit-learn tqdm pandas python-Levenshtein path colorama matplotlib seaborn

In [3]:
# !pip install python-Levenshtein

In [4]:
# @title Download git repo
# !git clone https://github.com/marquesarthur/vanilla-bert-vs-huggingface.git

In [5]:
# %cd vanilla-bert-vs-huggingface
# !git pull
# !ls -l

In [6]:
# @title Import data as JSON
import itertools
import json
import logging
import os
import sys
import random
from pathlib import Path

from Levenshtein import ratio
from colorama import Fore, Style

logger = logging.getLogger()
logger.level = logging.DEBUG
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

from ds_android import get_input_for_BERT

raw_data = get_input_for_BERT()

print('Sample entry from data:')
print(json.dumps(raw_data[0], indent=4, sort_keys=True))

39 129  https://developer.android.com/training/permissions/requesting
14 21  https://stackoverflow.com/questions/5233543
4 34  https://github.com/morenoh149/react-native-contacts/issues/516
27 63  https://guides.codepath.com/android/Understanding-App-Permissions
9 161  https://www.avg.com/en/signal/guide-to-android-app-permissions-how-to-use-them-smartly
9 15  https://developer.android.com/training/volley/request
14 65  https://stackoverflow.com/questions/28504524
20 59  https://medium.com/@JasonCromer/android-asynctask-http-request-tutorial-6b429d833e28
5 97  https://www.twilio.com/blog/5-ways-to-make-http-requests-in-java
4 12  https://stackoverflow.com/questions/33241952
6 33  https://github.com/realm/realm-java/issues/776
3 17  https://stackoverflow.com/questions/8712652
8 59  https://dzone.com/articles/android-rotate-and-scale
5 470  https://developer.android.com/reference/android/widget/TextView
7 11  https://stackoverflow.com/questions/19025301
8 95  https://docs.oracle.com/java

4 54  https://developer.android.com/training/gestures/scroll
4 16  https://stackoverflow.com/questions/39588322
20 196  https://developer.android.com/training/dependency-injection/dagger-android
6 44  https://stackoverflow.com/questions/57235136
24 121  https://guides.codepath.com/android/dependency-injection-with-dagger-2
Sample entry from data:
{
    "category_index": 1,
    "question": "Permission Denial when trying to access contacts in Android",
    "source": "https://developer.android.com/training/permissions/requesting",
    "text": "Every Android app runs in a limited-access sandbox.",
    "weights": 1
}


In [7]:
from collections import Counter, defaultdict

cnt = Counter([d['category_index'] for d in raw_data])

total = sum(cnt.values())

labels_cnt = [cnt[0] / float(total), cnt[1] / float(total)]
print('label distribution')
print('')
print('not-relevant -- {:.0f}%'.format(labels_cnt[0] * 100))
print('RELEVANT ------ {:.0f}%'.format(labels_cnt[1] * 100))

label distribution

not-relevant -- 88%
RELEVANT ------ 12%


In [8]:
seframes = {}
with open('seframes.json') as input_file:
    seframes = json.load(input_file)

In [9]:
def has_meaningful_frame(text):    
    meaning_frames = [
        'Temporal_collocation', 'Execution', 'Using', 'Intentionally_act',
        'Being_obligated', 'Likelihood', 'Causation', 'Required_event',
        'Desiring', 'Awareness', 'Grasp', 'Attempt'
    ]
    
    if text in seframes:
        text_labels = seframes[text]
        if any([elem in meaning_frames for elem in text_labels]):
            return True
        
    return False

In [10]:
fold_results = dict()
if os.path.isfile('bert_ds_android_pyramid.json'):
    logger.info(Fore.YELLOW + "Loading data from cache" + Style.RESET_ALL)
    with open('bert_ds_android_pyramid.json') as input_file:
        fold_results = json.load(input_file)

Loading data from cache


In [11]:
# @title Set environment variables

model_id = 'bert-base-uncased'
# model_id = 'distilbert-base-uncased'

import os
import contextlib
import tensorflow as tf
import os
import codecs
import numpy as np
import math
import json

import numpy as np
import pandas as pd

from collections import defaultdict, Counter
from tqdm import tqdm

USE_TPU = False
os.environ['TF_KERAS'] = '1'

# @title Initialize TPU Strategy
if USE_TPU:
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
    tf.contrib.distribute.initialize_tpu_system(resolver)
    strategy = tf.contrib.distribute.TPUStrategy(resolver)

# sklearn libs
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

# Tensorflow Imports
import tensorflow as tf
from tensorflow.python import keras
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers


# Hugging face imports
from transformers import AutoTokenizer
from transformers import TFDistilBertForSequenceClassification, TFBertForSequenceClassification
from transformers import TFDistilBertModel, DistilBertConfig
from transformers import DistilBertTokenizerFast, BertTokenizerFast

Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


In [12]:
# @title Model parameters

# Bert Model Constants
SEQ_LEN = 64 # 128
BATCH_SIZE = 64 # 64 32 larger batch size causes OOM errors
EPOCHS = 10 # 3 4
LR = 1e-5 # 2e-5

# 3e-4, 1e-4, 5e-5, 3e-5
# My own constants
# USE_FRAME_FILTERING = False
# UNDERSAMPLING = True
# N_UNDERSAMPLING = 2 # ratio of how many samples from 0-class, to 1-class, e.g.: 2:1
# USE_DS_SYNTHETIC = False

USE_FRAME_FILTERING = False
UNDERSAMPLING = True
N_UNDERSAMPLING = 2 # ratio of how many samples from 0-class, to 1-class, e.g.: 2:1
USE_DS_SYNTHETIC = False
MIN_W = 3

In [13]:
# @title JSON to dataframe helper functions
def undersample_df(df, n_times=3):
    class_0,class_1 = df.category_index.value_counts()
    c0 = df[df['category_index'] == 0]
    c1 = df[df['category_index'] == 1]
    df_0 = c0.sample(int(n_times * class_1))
    
    undersampled_df = pd.concat([df_0, c1],axis=0)
    return undersampled_df

def get_ds_synthetic_data(min_w=MIN_W):
    short_task = {
      "bugzilla": """How to query bugs using the custom fields with the Bugzilla REST API?""",
      "databases": """Which technology should be adopted for the database layer abstraction: Object/Relational Mapping (ORM) or a Java Database Connectivity API (JDBC)?""",
      "gpmdpu": """Can I bind the cmd key to the GPMDPU shortcuts?""",
      "lucene": """How does Lucene compute similarity scores for the BM25 similarity?""",
      "networking": """Which technology should be adopted for the notification system, Server-Sent Events (SSE) or WebSockets?""",
    }

    with open('relevance_corpus.json') as ipf:
        aux = json.load(ipf)
        raw_data = defaultdict(list)
        for d in aux:
            if d['task'] == 'yargs':
                continue

            raw_data['text'].append(d['text'])
            raw_data['question'].append(short_task[d['task']])
            raw_data['source'].append(d['source'])
            raw_data['category_index'].append(1 if d['weight'] > min_w else 0)
            raw_data['weights'].append(d['weight'] if d['weight'] > min_w else 0)
 
        data = pd.DataFrame.from_dict(raw_data)
        data = undersample_df(data, n_times=1)
        data = data.sample(frac=1).reset_index(drop=True)
      
    return data

def get_class_weights(y, smooth_factor=0, upper_bound=5.0):
    """
    Returns the weights for each class based on the frequencies of the samples
    :param smooth_factor: factor that smooths extremely uneven weights
    :param y: list of true labels (the labels must be hashable)
    :return: dictionary with the weight for each class
    """
    counter = Counter(y)

    if smooth_factor > 0:
        p = max(counter.values()) * smooth_factor
        for k in counter.keys():
            counter[k] += p

    majority = max(counter.values())

    clazz = {cls: float(majority / count) for cls, count in counter.items()}
    result = {}
    for key, value in clazz.items():
        if value > upper_bound:
            value = upper_bound
        
        result[key] = value
    return result

def add_raw_data(result, data):
    s = data['source']
    if 'docs.oracle' in s or 'developer.android' in s:
        source_type = 'api'
    elif 'stackoverflow.com' in s:
        source_type = 'so'
    elif 'github.com' in s:
        source_type = 'git'
    else:
        source_type = 'misc'
    pyramid = 1 if data['weights'] > 1 else 0
    
    result['text'].append(data['text'])
    result['question'].append(data['question'])
    result['source'].append(data['source'])
    result['category_index'].append(pyramid)
    result['weights'].append(data['weights'])
    result['source_type'].append(source_type)


In [14]:
# @title Tokenizer

print(model_id)
if model_id == 'distilbert-base-uncased':
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_id, cache_dir='/home/msarthur/scratch', local_files_only=True)
else:
    tokenizer = BertTokenizerFast.from_pretrained(model_id, cache_dir='/home/msarthur/scratch', local_files_only=True)

bert-base-uncased


In [15]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [16]:
# @title data encoder

def _encode(tokenizer, dataframe, max_length=SEQ_LEN):
    
    seq_a = dataframe['text'].tolist()
    seq_b = dataframe['question'].tolist()
    
    return tokenizer(seq_a, seq_b, truncation=True, padding=True, max_length=max_length)

def to_one_hot_encoding(data, nb_classes = 2):
    targets = np.array([data]).reshape(-1)
    one_hot_targets = np.eye(nb_classes)[targets]
    return one_hot_targets    

In [17]:
# @title Metrics & Logging functions

from sklearn.metrics import classification_report

recommendation_metrics = defaultdict(list)
prediction_metrics = defaultdict(list)
api_metrics = defaultdict(list)
so_metrics = defaultdict(list)
git_metrics = defaultdict(list)
misc_metrics = defaultdict(list)

classification_report_lst = []
log_examples_lst = []
source_lst = []
venn_diagram_set = []

def aggregate_macro_metrics(store_at, precision, recall, fscore):   
    store_at['precision'].append(precision)
    store_at['recall'].append(recall)
    store_at['fscore'].append(fscore)
    
    
def aggregate_macro_source_metrics(precision, recall, fscore, source):
    s = source
    if 'docs.oracle' in s or 'developer.android' in s:
        aggregate_macro_metrics(api_metrics, precision, recall, fscore)
    elif 'stackoverflow.com' in s:
        aggregate_macro_metrics(so_metrics, precision, recall, fscore)
    elif 'github.com' in s:
        aggregate_macro_metrics(git_metrics, precision, recall, fscore)        
    elif  'github.com' not in s and 'docs.oracle' not in s and 'developer.android' not in s and 'stackoverflow.com' not in s:
        aggregate_macro_metrics(misc_metrics, precision, recall, fscore)
    

def aggregate_recommendation_metrics(store_at, k, precision_at_k, pyramid_precision_at_k):
    store_at['k'].append(k)
    store_at['precision'].append(precision_at_k)
    store_at['∆ precision'].append(pyramid_precision_at_k)
    
def aggregate_report_metrics(clz_report):
    relevant_label = str(1)
    if relevant_label in clz_report:
        for _key in ['precision', 'recall']:
            if _key in clz_report[relevant_label]:
                clz_report_lst[_key].append(clz_report[relevant_label][_key])    
                
def log_examples(task_title, source, text, pweights, y_predict, y_probs, k=10):
    # get the predicted prob at every index
    idx_probs = [(idx, y_predict[idx], y_probs[idx]) for idx, _ in enumerate(y_predict)]
    
    # filter probs for all indexes predicted as relevant  
    idx_probs = list(filter(lambda k: k[1] == 1, idx_probs))
    
    most_probable = sorted(idx_probs, key=lambda i: i[2], reverse=True)
    
    result = [idx for idx, _, _ in most_probable][:k]
    
    for idx in result:
        log_examples_lst.append((
            source, 
            task_title,
            pweights[idx],
            y_predict[idx],
            y_probs[idx],
            text[idx]
        ))
        
def log_venn_diagram(y_true, y_predicted, text):
    cnt = 0
    try:
        for _true, _predict, _t in zip(y_true, y_predicted, text):
            if _true == 1 and _predict == 1:
                cnt += 1
                venn_diagram_set.append(_t)
    except Exception as ex:
        logger.info(str(ex))
    logger.info(Fore.RED + str(cnt) + Style.RESET_ALL + " entries logged")

    
def avg_macro_metric_for(data):
    __precision = data['precision']
    __recall = data['recall']
    __fscore = data['fscore']

    return np.mean(__precision), np.mean(__recall), np.mean(__fscore)        

In [18]:
#@title Training procedures

def get_train_val_test(task_uid, size=0.9, undersample=False, aug=True, undersample_n=3):
    if not isinstance(task_uid, list):
        task_uid = [task_uid]
        
    train_data_raw = defaultdict(list)
    test_data_raw = defaultdict(list)
    
    for _data in tqdm(CORPUS):
        if _data['question'] in task_uid:
            add_raw_data(test_data_raw, _data)
        else:
            add_raw_data(train_data_raw, _data)
    
    train_val = pd.DataFrame.from_dict(train_data_raw)
    test = pd.DataFrame.from_dict(test_data_raw)
    
    # https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    #  randomize rows....    
    train_val = train_val.sample(frac=1).reset_index(drop=True)
    test = test.sample(frac=1).reset_index(drop=True)
    
    if undersample:
        train_val = undersample_df(train_val, n_times=undersample_n)
        train_val = train_val.sample(frac=1).reset_index(drop=True)
        
    if aug:
        train_val = pd.concat([train_val, get_ds_synthetic_data()],axis=0)
        train_val = train_val.sample(frac=1).reset_index(drop=True)
    
    weights = get_class_weights(train_val['category_index'].tolist())
    
    train, val = train_test_split(
        train_val, 
        stratify=train_val['category_index'].tolist(), 
        train_size=size
    )
    
    return train, val, test, weights        

In [19]:
def update_predictions(task_title, text, y_predict, y_probs, relevant_class=1):
    result = []
    
    for _t, _y, _prob in zip(text, y_predict, y_probs):
        if _y == relevant_class:
            if has_meaningful_frame(_t):
                result.append(_y)
            else:
                result.append(0)
        else:
            result.append(_y)
    
    return result    

In [20]:
# @title Testing procedures

# https://medium.com/geekculture/hugging-face-distilbert-tensorflow-for-custom-text-classification-1ad4a49e26a7
def eval_model(model, test_data):
    preds = model.predict(test_data.batch(1)).logits  
    
    #transform to array with probabilities
    res = tf.nn.softmax(preds, axis=1).numpy()      

    return res.argmax(axis=-1), res[:, 1]

def test_model(source, df_test, model, tokenizer, pos_filter=False):
    
    df_source = df_test[df_test["source"] == source]   
    task_title = df_source['question'].tolist()[0]
    text = df_source['text'].tolist()
    pweights = df_source['weights'].tolist()
    
    # Encode X_test
    test_encodings = _encode(tokenizer, df_source)
    test_labels = df_source['category_index'].tolist()
    
    test_dataset = tf.data.Dataset.from_tensor_slices((
        dict(test_encodings),
        test_labels
    ))
    
    y_true = [y.numpy() for x, y in test_dataset]
    
    # <= 0  means that an artifact has no relevant information highlighted 
    # by two or more annotators. these artifacts are ignored
    if len(list(filter(lambda k: k == 1, y_true))) > 0:
        y_predict, y_probs = eval_model(model, test_dataset)

        if pos_filter:
            y_predict = update_predictions(task_title, text, y_predict, y_probs)


        accuracy = accuracy_score(y_true, y_predict)
        macro_f1 = f1_score(y_true, y_predict, average='macro')

        classification_report_lst.append(classification_report(y_true, y_predict))
        aggregate_report_metrics(classification_report(y_true, y_predict, output_dict=True))


        logger.info("-" * 20)    

        logger.info("Y")
        logger.info("[0s] {} [1s] {}".format(
            len(list(filter(lambda k: k== 0, y_true))),
            len(list(filter(lambda k: k== 1, y_true)))
        ))


        logger.info("predicted")
        logger.info("[0s] {} [1s] {}".format(
            len(list(filter(lambda k: k== 0, y_predict))),
            len(list(filter(lambda k: k== 1, y_predict)))
        ))

        logger.info("-" * 20)

        logger.info("Accuracy: {:.4f}".format(accuracy))
        logger.info("macro_f1: {:.4f}".format(macro_f1))

        precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_predict, average='macro')

        aggregate_macro_metrics(prediction_metrics, precision, recall, fscore)
        aggregate_macro_source_metrics(precision, recall, fscore, source)

        logger.info("Precision: {:.4f}".format(precision))
        logger.info("Recall: {:.4f}".format(recall))
        logger.info("F1: {:.4f}".format(fscore))

        log_examples(task_title, source, text, pweights, y_predict, y_probs, k=10)
        log_venn_diagram(y_true, y_predict, text)
        source_lst.append(source)

In [21]:
def add_idx_fold_results(idx_split, store_at):
    if idx_split not in store_at:
        store_at[idx_split] = dict()
        store_at[idx_split]['run_cnt'] = 0
        store_at[idx_split]['overall'] = defaultdict(list)
        store_at[idx_split]['api'] = defaultdict(list)
        store_at[idx_split]['so'] = defaultdict(list)
        store_at[idx_split]['git'] = defaultdict(list)
        store_at[idx_split]['misc'] = defaultdict(list)
    
    store_at[idx_split]['run_cnt'] += 1
    
    _precision, _recall, _f1score = avg_macro_metric_for(prediction_metrics)
    store_at[idx_split]['overall']['precision'].append(_precision)
    store_at[idx_split]['overall']['recall'].append(_recall)
    store_at[idx_split]['overall']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(api_metrics)
    store_at[idx_split]['api']['precision'].append(_precision)
    store_at[idx_split]['api']['recall'].append(_recall)
    store_at[idx_split]['api']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(so_metrics)
    store_at[idx_split]['so']['precision'].append(_precision)
    store_at[idx_split]['so']['recall'].append(_recall)
    store_at[idx_split]['so']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(git_metrics)
    store_at[idx_split]['git']['precision'].append(_precision)
    store_at[idx_split]['git']['recall'].append(_recall)
    store_at[idx_split]['git']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(misc_metrics)
    store_at[idx_split]['misc']['precision'].append(_precision)
    store_at[idx_split]['misc']['recall'].append(_recall)
    store_at[idx_split]['misc']['fscore'].append(_f1score)  

In [22]:
# model = TFBertForSequenceClassification.from_pretrained(model_id, cache_dir='/home/msarthur/scratch', local_files_only=True)

In [23]:
# @title 10-fold cross validation WIP
CORPUS = raw_data

all_tasks = sorted(list(set([d['question'] for d in raw_data])))
rseed = 20210343
random.seed(rseed)
random.shuffle(all_tasks)

from sklearn.model_selection import KFold


file_handler = logging.FileHandler('/home/msarthur/scratch/LOG-bert_ds_android_pyramid.ans')
file_handler.setLevel(logging.DEBUG)
logger.addHandler(file_handler)


n_splits = 10
kf = KFold(n_splits=n_splits, random_state=rseed)
np_tasks_arr = np.array(all_tasks)



idx_split = 0
for train_index, test_index in kf.split(np_tasks_arr):

    idx_split = str(idx_split)
    eval_fold = True
    # 10 runs per fold to avoid reporting peek results in a given fold
    if idx_split in fold_results and fold_results[idx_split]['run_cnt'] >= 10:
        logger.info(Fore.RED + f"Fold {idx_split} FULLY TESTED" + Style.RESET_ALL)
        eval_fold = False


    if eval_fold:
        # <------------------------------------------------------------------------- EVAL VARIABLES
        recommendation_metrics = defaultdict(list)
        prediction_metrics = defaultdict(list)
        api_metrics = defaultdict(list)
        so_metrics = defaultdict(list)
        git_metrics = defaultdict(list)
        misc_metrics = defaultdict(list)
        random_prediction_metrics = defaultdict(list)
        clz_report_lst = defaultdict(list)

        classification_report_lst = []
        log_examples_lst = []
        source_lst = []
        venn_diagram_set = []
        # <------------------------------------------------------------------------- EVAL VARIABLES


        test_tasks_lst = np_tasks_arr[test_index].tolist()

        logger.info("")
        logger.info(Fore.RED + f"Fold {idx_split}" + Style.RESET_ALL)
        logger.info('\n'.join(test_tasks_lst))

        # <------------------------------------------------------------------------- INPUT
        df_train, df_val, df_test, weights = get_train_val_test(
            test_tasks_lst,
            aug=USE_DS_SYNTHETIC,
            undersample=UNDERSAMPLING, 
            undersample_n=N_UNDERSAMPLING
        )
        # <------------------------------------------------------------------------- INPUT

        logger.info('-' * 10)
        logger.info(Fore.RED + 'train'+ Style.RESET_ALL)
        logger.info(str(df_train.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'test'+ Style.RESET_ALL)
        logger.info(str(df_test.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'weights'+ Style.RESET_ALL)
        logger.info(str(weights))
        logger.info('-' * 10)


        # Encode X_train
        train_encodings = _encode(tokenizer, df_train)
        train_labels = df_train['category_index'].tolist()

        # Encode X_valid
        val_encodings = _encode(tokenizer, df_val)
        val_labels = df_val['category_index'].tolist()


        # https://huggingface.co/transformers/custom_datasets.html
        train_dataset = tf.data.Dataset.from_tensor_slices((
            dict(train_encodings),
            train_labels
        ))

        val_dataset = tf.data.Dataset.from_tensor_slices((
            dict(val_encodings),
            val_labels
        ))


        if model_id == 'distilbert-base-uncased':
            model = TFDistilBertForSequenceClassification.from_pretrained(
                model_id, cache_dir='/home/msarthur/scratch'
            )
        else:
            model = TFBertForSequenceClassification.from_pretrained(
                model_id, cache_dir='/home/msarthur/scratch', local_files_only=True
            )

        # freeze all the parameters
        # for param in model.parameters():
        #   param.requires_grad = False


        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        METRICS = [
            tf.keras.metrics.SparseCategoricalAccuracy()
        ]

        early_stopper = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', mode='min', patience=4, 
            verbose=1, restore_best_weights=True
        )

        # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
        checkpoint_filepath = '/home/msarthur/scratch/best_pyramid_model'

        mc = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, 
            monitor='val_loss', mode='min', verbose=1, 
            save_best_only=True,
            save_weights_only=True
        )

        model.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=METRICS
        )

        # https://discuss.huggingface.co/t/how-to-dealing-with-data-imbalance/393/3
        # https://wandb.ai/ayush-thakur/huggingface/reports/Early-Stopping-in-HuggingFace-Examples--Vmlldzo0MzE2MTM
        model.fit(
            train_dataset.shuffle(1000).batch(BATCH_SIZE), 
            epochs=EPOCHS, 
            batch_size=BATCH_SIZE,
            class_weight=weights,
            validation_data=val_dataset.shuffle(1000).batch(BATCH_SIZE),
            callbacks=[early_stopper, mc]
        )

        model.load_weights(checkpoint_filepath)

        logger.info("")
        logger.info(Fore.RED + f"Testing model" + Style.RESET_ALL)
        for source in df_test["source"].unique():
            df_source = df_test[df_test["source"] == source]   
            logger.info(source)
            test_model(source, df_source, model, tokenizer, pos_filter=USE_FRAME_FILTERING)

        add_idx_fold_results(idx_split, fold_results)
        if 'venn_diagram_set' not in fold_results:
            fold_results['venn_diagram_set'] = []

        fold_results['venn_diagram_set'] += venn_diagram_set
        fold_results['venn_diagram_set'] = list(set(fold_results['venn_diagram_set']))


        _precision, _recall, _f1score = avg_macro_metric_for(prediction_metrics)

        logger.info("")
        logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
        logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
        logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
        logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)




        log_sources_data = [api_metrics, so_metrics, git_metrics, misc_metrics]
        log_sources_ids = ['api_metrics', 'so_metrics', 'git_metrics', 'misc_metrics']

        for _id, __data in zip(log_sources_ids, log_sources_data):
            _precision, _recall, _f1score = avg_macro_metric_for(__data)

            logger.info("")
            logger.info(Fore.YELLOW + f"{_id}" + Style.RESET_ALL)
            logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
            logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
            logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


    idx_split = int(idx_split)
    idx_split += 1
    logger.info(f"next {idx_split}")
#     break
#         if idx_split >= 7:
#             logger.info(f"breaking at {idx_split}")
#             break


Fold 0
how can i get the value of text view in recyclerview item?
Hide MarkerView when nothing selected
How to check programmatically whether app is running in debug mode or not?
JSONObject parse dictionary objects
Want to add drawable icons insteadof colorful dots


/home/msarthur/hface/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
100%|██████████| 7930/7930 [00:00<00:00, 801504.43it/s]

----------
train
0    504
1    252
Name: category_index, dtype: int64

test
0    708
1     27
Name: category_index, dtype: int64

weights
{1: 2.0, 0: 1.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x2ae4bdabd3d0>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expe

...

/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 7930/7930 [00:00<00:00, 810903.55it/s]

----------
train
0    507
1    254
Name: category_index, dtype: int64

test
0    553
1     25
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 2.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
12/12 [==============================] - ETA: 0s - loss: 0.9426 - sparse_categorical_accuracy: 0.6334The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output

/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [24]:
for key_i, value in fold_results.items():
    if isinstance(value, dict):
        for key_j, __data in value.items():
            if key_j == 'overall':
                logger.info(Fore.YELLOW + f"{key_i}" + Style.RESET_ALL)
                logger.info("precision: " + Fore.RED +
                            "{:.3f}".format(np.mean(__data['precision'])) + Style.RESET_ALL +
                           f" {str([round(x, 2) for x in __data['precision']])}")
                logger.info("recall:    " + Fore.RED +
                            "{:.3f}".format(np.mean(__data['recall'])) + Style.RESET_ALL+
                           f" {str([round(x, 2) for x in __data['recall']])}")
                logger.info("f1-score:  " + 
                            Fore.RED + "{:.3f}".format(np.mean(__data['fscore'])) + Style.RESET_ALL+
                           f" {str([round(x, 2) for x in __data['fscore']])}")

0
precision: 0.524 [0.48, 0.54, 0.56]
recall:    0.601 [0.56, 0.58, 0.66]
f1-score:  0.493 [0.5, 0.51, 0.47]
1
precision: 0.600 [0.65, 0.54, 0.62]
recall:    0.651 [0.71, 0.6, 0.65]
f1-score:  0.506 [0.53, 0.46, 0.52]
2
precision: 0.561 [0.55, 0.57, 0.57]
recall:    0.626 [0.58, 0.67, 0.63]
f1-score:  0.466 [0.48, 0.4, 0.51]
3
precision: 0.511 [0.5, 0.5, 0.54]
recall:    0.537 [0.52, 0.55, 0.54]
f1-score:  0.451 [0.48, 0.43, 0.44]
4
precision: 0.567 [0.62, 0.56, 0.52]
recall:    0.660 [0.69, 0.64, 0.65]
f1-score:  0.520 [0.56, 0.47, 0.53]
5
precision: 0.507 [0.51, 0.5, 0.51]
recall:    0.622 [0.63, 0.61, 0.63]
f1-score:  0.431 [0.43, 0.43, 0.43]
6
precision: 0.520 [0.55, 0.48, 0.53]
recall:    0.592 [0.63, 0.5, 0.64]
f1-score:  0.448 [0.38, 0.49, 0.48]
7
precision: 0.604 [0.59, 0.58, 0.64]
recall:    0.668 [0.64, 0.68, 0.69]
f1-score:  0.539 [0.5, 0.52, 0.6]
8
precision: 0.551 [0.59, 0.53, 0.53]
recall:    0.702 [0.79, 0.7, 0.61]
f1-score:  0.487 [0.58, 0.45, 0.43]
9
precision: 0.524 [

In [25]:
logger.info(Fore.YELLOW + "Caching results" + Style.RESET_ALL)
with open('bert_ds_android_pyramid.json', 'w') as fo:
    json.dump(fold_results, fo, indent=4)

Caching results


In [26]:
fold_results.keys()

dict_keys(['0', 'venn_diagram_set', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [27]:
# cnt = 0
# for source in df_test["source"].unique():
#     df_source = df_test[df_test["source"] == source]   
#     logger.info(source)
#     test_model(source, df_source, model, tokenizer, pos_filter=True)
#     cnt += 1
#     if cnt >= 5:
#         break

In [28]:
#@title Metrics report
# logger.info(json.dumps(fold_results, indent=4, sort_keys=True))

In [29]:
# _precision, _recall, _f1score = avg_macro_metric_for(prediction_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


# _precision, _recall, _f1score = avg_macro_metric_for(api_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "API metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

# _precision, _recall, _f1score = avg_macro_metric_for(so_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "SO metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

# _precision, _recall, _f1score = avg_macro_metric_for(git_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "GIT metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

# _precision, _recall, _f1score = avg_macro_metric_for(misc_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "MISC metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

In [30]:
def examples_per_source_type(source_type='misc', n_samples=None):
    _sources = list(set([x[0] for x in log_examples_lst]))

    _template = "[w={}]" + Fore.RED + "[y={}]" + Fore.YELLOW + "[p={:.4f}]" + Style.RESET_ALL + " {}"

    idx = 0
    for s in _sources:
        examples_in_source = []
        if source_type == 'api' and ('docs.oracle' in s or 'developer.android' in s):
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]
            idx += 1
        elif source_type == 'so' and ('stackoverflow.com' in s):
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]            
            idx += 1
        elif source_type == 'git' and ('github.com' in s):
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]
            idx += 1
        elif source_type == 'misc' and 'github.com' not in s and 'docs.oracle' not in s and 'developer.android' not in s and 'stackoverflow.com' not in s:
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]
            idx += 1
        if not examples_in_source:
            continue
        logger.info('')
        logger.info(Fore.RED + f"{task_title}" + Style.RESET_ALL)    
        logger.info(s)
        logger.info('')

        for _, _, pweights, y_predict, y_probs, text in examples_in_source:
            logger.info(_template.format(pweights, y_predict, y_probs, text))
            logger.info('')
        logger.info('-' * 20)
      
        if n_samples and idx >= n_samples:
            break
    

In [31]:
#@title Sample prediction outputs for API sources

logger.info(Fore.RED + "API" + Style.RESET_ALL)
examples_per_source_type(source_type='api', n_samples=8)

API

Hilt: How to prevent Hilt from picking dependency from a library?
https://developer.android.com/training/dependency-injection/hilt-android

[w=0][y=1][p=0.8685] To migrate a project that uses Dagger to Hilt, see the migration guide and the Migrating your Dagger app to Hilt codelab.

[w=0][y=1][p=0.8637] To learn more about Hilt component scopes, see Scoping in Android and Hilt.

[w=0][y=1][p=0.8604] The following example demonstrates how to scope a binding to a component in a Hilt module.

[w=0][y=1][p=0.8563] The Hilt module AnalyticsModule is annotated with @InstallIn ( ActivityComponent:: class ) because you want Hilt to inject that dependency into ExampleActivity.

[w=0][y=1][p=0.8557] If you want a content provider to use Hilt to get some dependencies, you need to define an interface that is annotated with @EntryPoint for each binding type that you want and include qualifiers.

[w=0][y=1][p=0.8525] @AndroidEntryPoint generates an individual Hilt component for each Android cla

In [32]:
#@title Sample prediction outputs for GIT sources

logger.info(Fore.RED + "GIT" + Style.RESET_ALL)
examples_per_source_type(source_type='git', n_samples=8)

GIT


In [33]:
#@title Sample prediction outputs for SO sources

logger.info(Fore.RED + "SO" + Style.RESET_ALL)
examples_per_source_type(source_type='so', n_samples=8)

SO

Android SQLite performance in complex queries
https://stackoverflow.com/questions/4015026

[w=0][y=1][p=0.8946] I dropped this into my ContentProvider.query -LRB- -RRB- and now I can see exactly how all the queries are getting performed.

[w=3][y=1][p=0.8892] Use EXPLAIN QUERY PLAN on your queries to see which index would be used or if the query requires a full table scan.

[w=2][y=1][p=0.8866] Here's a bit of code to get EXPLAIN QUERY PLAN results into Android logcat from a running Android app.

[w=3][y=1][p=0.8780] Pin down exactly which queries you need to optimize.

[w=3][y=1][p=0.8738] If you have more complex queries that can't make use of any indexes that you might create, you can de-normalize your schema, structuring your data in such a way that the queries are simpler and can be answered using indexes.

[w=3][y=1][p=0.8725] For SELECTs and UPDATEs, indexes can things up, but only if the indexes you create can actually be used by the queries that you need speeding up.

[w=0

In [34]:
#@title Sample prediction outputs for MISC sources

logger.info(Fore.RED + "MISC" + Style.RESET_ALL)
examples_per_source_type(source_type='misc', n_samples=8)

MISC

Hilt: How to prevent Hilt from picking dependency from a library?
https://prog.world/a-practical-guide-to-using-hilt-with-kotlin

[w=0][y=1][p=0.8682] Setting up Hilt To set up Hilt in your application, first follow the directions from the guide: Installing Gradle Build ...

[w=0][y=1][p=0.8544] An easy way to use dependency injection in Android apps Hilt Is a new library for dependency injection built on top of Dagger ... It allows you to use Dagger's capabilities in Android apps in a simplified way.

[w=2][y=1][p=0.8510] How to make a dependency injectable To make an object embeddable in Hilt, you need to tell Hilt how to instantiate that object.

[w=0][y=1][p=0.8418] After installing all the required elements and plugins to use Hilt, annotate your Application class @HiltAndroidApp.

[w=0][y=1][p=0.8400] These classes are entry points into the Hilt dependency graph, and Hilt needs to know that they have dependencies to inject.

[w=0][y=1][p=0.8387] Hilt module Think of it as a 

In [35]:
logger.info(Fore.RED + f"{len(fold_results['venn_diagram_set'])} entries VENN SET" + Style.RESET_ALL)
for _t in fold_results['venn_diagram_set']:
    logger.info(_t)

247 entries VENN SET
To do this, we need to create our own custom ArrayAdapter class.
To be able to record, your app must tell the user that it will access the device's audio input.
Use ANALYZE to allow SQLite's query planner to work more efficiently.
Returns the value mapped by name if it exists and is an int or can be coerced to an int, or throws otherwise.
But that does not keep your site safe from HTTP client request forgery.
It triggers when you scroll the view -LRB- list for example -RRB-.
basically I used a private string property taken as a string to construct a computed property with that enum correspondance.
dataSpec.length is the length of data that the caller wants to read, or C.LENGTH _ UNSET to read to the end of the media.
I haven't tried recording phone call's but there is a option in LINK for:
inject.Scope annotation - Dependencies declared with that scope with have caching Provider with double-check lock generated and only single instance will be created for it within

The following code shows how to do this in the context of an Activity, but this is also possible from within a Fragment.
There are a few different ways of using it, so let's look at an example.
These repeated calls can seriously harm the ListView's performance, especially if your app is running on limited resources and/or you have a very large list.
But these action buttons should not duplicate the action performed when the user taps the notification.
So ideally we also would have an easy way of processing an array of businesses into an ArrayList of Business objects.
This code sample demonstrates how to modify the contents of the text view defined in the previous XML layout:
This class implements SurfaceHolder.Callback in order to capture the callback events for creating and destroying the view, which are needed for assigning the camera preview input.
Beginning in Android 6.0 -LRB- API level 23 -RRB-, users grant permissions to apps while the app is running, not when they install the a

Dangerous permission groups, however, can give apps access to things like your calling history, private messages, location, camera, microphone, and more.
Users can use the system settings to choose the level of detail visible in lock screen notifications, including the option to disable all lock screen notifications.
To register a key pair for use with the SafetyNet reCAPTCHA API, navigate to the reCAPTCHA Android signup site, then complete the following sequence of steps:
Usage Defining a Fragment A fragment, like an activity, has an XML layout file and a Java class that represents the Fragment controller.
It returns View that will be displayed as your list/grid/gallary / any view that use adapter item.
Below we place the toolbar at the top of a LinearLayout like the standard ActionBar:
A Toolbar is a generalization of action bars for use within application layouts.
I am using mic to record phone audio and also use the Telephony manager to find the calling state.
You may extend it fro

You also inflate a custom view from the XML layout defined in res/layout/list_item_recipe.xml -- more on this in the next section.
One thing that I ran across when I applied this approach to my production code is that you still need to keep the @JsonSubtypes annotation as part of the Base class.
Although what you are doing is really not appropriate.
After that, extend the class AsyncTask.
This line means: add a meta-property on serialization or read a meta-property on deserialization -LRB- include = JsonTypeInfo.As.PROPERTY -RRB- called'' @class'' -LRB- property ='' @class'' -RRB- that holds the fully-qualified Java class name -LRB- use = JsonTypeInfo.Id.CLASS -RRB-.
that is done because rotation is around the top left point ( the origin ) of the view.
If you have more complex queries that can't make use of any indexes that you might create, you can de-normalize your schema, structuring your data in such a way that the queries are simpler and can be answered using indexes.
Pass in your